In [1]:
#Définition des librairies
library(jsonlite)
suppressPackageStartupMessages(library(Hmisc))
library(cluster)
library(Ecdat)
#library(compareGroups)
library(randomForest)
library(dplyr)

Loading required package: Ecfun

Attaching package: ‘Ecfun’

The following object is masked from ‘package:base’:

    sign


Attaching package: ‘Ecdat’

The following object is masked from ‘package:datasets’:

    Orange

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin


Attaching package: ‘dplyr’

The following object is masked from ‘package:randomForest’:

    combine

The following objects are masked from ‘package:Hmisc’:

    src, summarize

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
library(compareGroups)

Loading required package: gdata
gdata: read.xls support for 'XLS' (Excel 97-2004) files ENABLED.

gdata: read.xls support for 'XLSX' (Excel 2007+) files ENABLED.

Attaching package: ‘gdata’

The following objects are masked from ‘package:dplyr’:

    combine, first, last

The following object is masked from ‘package:randomForest’:

    combine

The following object is masked from ‘package:stats’:

    nobs

The following object is masked from ‘package:utils’:

    object.size

The following object is masked from ‘package:base’:

    startsWith

Loading required package: xtable

Attaching package: ‘xtable’

The following objects are masked from ‘package:Hmisc’:

    label, label<-

Loading required package: SNPassoc
Loading required package: haplo.stats
Loading required package: mvtnorm
Loading required package: parallel


In [3]:
#Chargement du jeu de données
path="/home/user/Documents/STA211-Datamining/Projet_STA/"
Airbnb<-fromJSON(paste(path,"airbnb_V3.json",sep=""))
sapply(Airbnb,class)
summary(Airbnb)


_id           accommodates              bathrooms 
             "integer"            "character"            "character" 
              bed_type               bedrooms                   beds 
           "character"            "character"            "character" 
         property_type              room_type         arrondissement 
           "character"            "character"              "numeric" 
host_identity_verified      host_is_superhost          host_listings 
           "character"            "character"            "character" 
         host_response             host_since          cancel_policy 
           "character"            "character"            "character" 
      instant_bookable        availability_30       availability_365 
           "character"              "integer"              "integer" 
       availability_60        availability_90           first_review 
             "integer"              "integer"            "character" 
           last_review             nb_reviews    score_communication 
           "character"            "character"              "numeric" 
        score_location            score_other      reviews_per_month 
             "numeric"              "numeric"              "numeric" 
           total_price            price_level 
             "numeric"            "character"

      _id           accommodates        bathrooms           bed_type        
 Min.   :    2525   Length:41094       Length:41094       Length:41094      
 1st Qu.: 3688220   Class :character   Class :character   Class :character  
 Median : 7887020   Mode  :character   Mode  :character   Mode  :character  
 Mean   : 8425822                                                           
 3rd Qu.:13304907                                                           
 Max.   :17968815                                                           
   bedrooms             beds           property_type       room_type        
 Length:41094       Length:41094       Length:41094       Length:41094      
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            

In [4]:

character_vars <- lapply(Airbnb, class) == "character"
Airbnb[, character_vars] <- lapply(Airbnb[, character_vars], as.factor)
#définition de la variable arrondissement comme qualitative
Airbnb$arrondissement <- as.factor(Airbnb$arrondissement ) 
sapply(Airbnb,class)

_id           accommodates              bathrooms 
             "integer"               "factor"               "factor" 
              bed_type               bedrooms                   beds 
              "factor"               "factor"               "factor" 
         property_type              room_type         arrondissement 
              "factor"               "factor"               "factor" 
host_identity_verified      host_is_superhost          host_listings 
              "factor"               "factor"               "factor" 
         host_response             host_since          cancel_policy 
              "factor"               "factor"               "factor" 
      instant_bookable        availability_30       availability_365 
              "factor"              "integer"              "integer" 
       availability_60        availability_90           first_review 
             "integer"              "integer"               "factor" 
           last_review             nb_reviews    score_communication 
              "factor"               "factor"              "numeric" 
        score_location            score_other      reviews_per_month 
             "numeric"              "numeric"              "numeric" 
           total_price            price_level 
             "numeric"               "factor"

In [5]:
Airbnb_train <- Airbnb %>% sample_frac(0.3)
disMat<-daisy(Airbnb_train,metric = "gower")

In [6]:
set.seed(123)
mixedClusters<-kmeans(disMat, centers=5)

In [7]:
table(mixedClusters$cluster)


   1    2    3    4    5 
2074 2176 2652 2615 2811 

In [8]:
Airbnb_train$cluster<-mixedClusters$cluster
Airbnb_train$cluster <- as.factor(Airbnb_train$cluster ) 

In [9]:
sapply(Airbnb_train,class)

_id           accommodates              bathrooms 
             "integer"               "factor"               "factor" 
              bed_type               bedrooms                   beds 
              "factor"               "factor"               "factor" 
         property_type              room_type         arrondissement 
              "factor"               "factor"               "factor" 
host_identity_verified      host_is_superhost          host_listings 
              "factor"               "factor"               "factor" 
         host_response             host_since          cancel_policy 
              "factor"               "factor"               "factor" 
      instant_bookable        availability_30       availability_365 
              "factor"              "integer"              "integer" 
       availability_60        availability_90           first_review 
             "integer"              "integer"               "factor" 
           last_review             nb_reviews    score_communication 
              "factor"               "factor"              "numeric" 
        score_location            score_other      reviews_per_month 
             "numeric"              "numeric"              "numeric" 
           total_price            price_level                cluster 
             "numeric"               "factor"               "factor"

In [17]:
group<-compareGroups(cluster~accommodates+bathrooms+bed_type+bedrooms+beds+property_type+room_type
                     +host_identity_verified+host_is_superhost+host_listings+host_response+host_since+cancel_policy
                     +instant_bookable+availability_365+availability_90+availability_60+availability_30
                     +first_review+last_review+nb_reviews+score_communication+score_location+score_other+reviews_per_month
                     +price_level+total_price
                     ,data = Airbnb_train)
clustab<-createTable(group)
clustab

Warning message in chisq.test(obj, simulate.p.value = TRUE):
“cannot compute simulated p-value with zero marginals”Warning message in chisq.test(obj, simulate.p.value = TRUE):
“Chi-squared approximation may be incorrect”


--------Summary descriptives table by 'cluster'---------

__________________________________________________________________________________________________ 
                             1            2            3            4            5       p.overall 
                           N=2074       N=2176       N=2652       N=2615       N=2811              
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯ 
accommodates:                                                                              0.000   
    T1                  167 (8.05%)   21 (0.97%)  130 (4.90%)   55 (2.10%)  133 (4.73%)            
    T2                  1257 (60.6%)  58 (2.67%)  1443 (54.4%) 1302 (49.8%) 1875 (66.7%)           
    T3                  233 (11.2%)  174 (8.00%)  267 (10.1%)  309 (11.8%)  257 (9.14%)            
    T4                  367 (17.7%)  844 (38.8%)  656 (24.7%)  846 (32.4%)  499 (17.8%)            
    T5+                  50 (2.41%)  1079